ASSIGNMENT GOES BELOW -

In [24]:
# --- Colab one-cell app: HF Summarizer + TTS + Export + Dark/Light toggle ---
# If you're in Colab, this should "just work".

!pip -q install -U transformers accelerate sentencepiece gradio gTTS pydub

import os, re, json, uuid, zipfile, datetime, tempfile, shutil
from pathlib import Path
import gradio as gr
from transformers import pipeline
from gtts import gTTS

# ========== CONFIG ==========
HF_TOKEN = "HFACE_TOKEN"  # <-- uses your provided token, no new resources created
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN
os.environ["HF_TOKEN"] = HF_TOKEN
# ============================

# Global state for the loaded pipeline
_SUMMARIZER = None
_CURRENT_MODEL_ID = None

def _load_model(model_id: str):
    """
    Load (or reload) a summarization pipeline from Hugging Face Hub.
    We pass token via env var so nothing is created server-side.
    """
    global _SUMMARIZER, _CURRENT_MODEL_ID
    if not model_id or "/" not in model_id:
        raise gr.Error("Please provide a valid Hugging Face model ID (e.g., 'facebook/bart-large-cnn').")
    try:
        # device_map="auto" will use GPU if available
        _SUMMARIZER = pipeline(
            "summarization",
            model=model_id,
            tokenizer=model_id,
            device_map="auto",
            # transformers>=4.41 reads token from env; passing explicit token is also okay:
            # token=os.environ.get("HF_TOKEN")
        )
        _CURRENT_MODEL_ID = model_id
        return gr.update(value=f"✅ Loaded model: `{model_id}`"), f"Model loaded: {model_id}"
    except Exception as e:
        _SUMMARIZER = None
        _CURRENT_MODEL_ID = None
        raise gr.Error(f"Model load failed: {e}")

def _chunk_text_words(text: str, max_words=800):
    """
    Naive chunker: splits long text into ~max_words pieces to avoid hitting model limits.
    Keeps paragraphs together where possible.
    """
    if len(text.split()) <= max_words:
        return [text]
    paras = re.split(r"\n\s*\n", text.strip())
    chunks, current = [], ""
    for p in paras:
        if not p.strip():
            continue
        if len((current + "\n\n" + p).split()) > max_words:
            if current.strip():
                chunks.append(current.strip())
            current = p
        else:
            current = (current + "\n\n" + p).strip()
    if current.strip():
        chunks.append(current.strip())
    return chunks

def _summarize(text, max_length, min_length, do_sample, temperature, top_p):
    """
    Summarize, chunking if needed. Returns summary text only.
    """
    if _SUMMARIZER is None:
        raise gr.Error("Please load a model first (top-left) before summarizing.")
    if not text or not text.strip():
        raise gr.Error("Please paste some input text.")

    pieces = _chunk_text_words(text, max_words=800)
    outputs = []
    for piece in pieces:
        result = _SUMMARIZER(
            piece,
            max_length=int(max_length),
            min_length=int(min_length),
            do_sample=bool(do_sample),
            temperature=float(temperature),
            top_p=float(top_p),
            truncation=True,
        )
        summary_piece = result[0]["summary_text"].strip()
        outputs.append(summary_piece)
    return "\n\n".join(outputs).strip()

def _tts(summary_text, lang="en"):
    """
    Convert summary to speech (MP3) using gTTS. Returns file path.
    """
    if not summary_text or not summary_text.strip():
        raise gr.Error("Nothing to read. Please summarize first.")
    fn = Path(tempfile.gettempdir()) / f"summary_{uuid.uuid4().hex}.mp3"
    tts = gTTS(summary_text, lang=lang)
    tts.save(str(fn))
    return str(fn)

def _export(original_text, summary_text, model_id, audio_path):
    """
    Export a ZIP containing summary.txt, meta.json, and the audio MP3 (if present).
    Returns path to the ZIP.
    """
    if not summary_text or not summary_text.strip():
        raise gr.Error("No summary to export. Please summarize first.")

    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    base = Path(tempfile.gettempdir()) / f"summarization_{ts}_{uuid.uuid4().hex[:6]}"
    base.parent.mkdir(parents=True, exist_ok=True)

    txt_path = base.with_suffix(".txt")
    meta_path = base.with_suffix(".json")
    zip_path = base.with_suffix(".zip")

    meta = {
        "timestamp": ts,
        "model_id": model_id or _CURRENT_MODEL_ID,
        "original_chars": len(original_text or ""),
        "summary_chars": len(summary_text or ""),
    }

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(summary_text)

    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
        z.write(txt_path, arcname="summary.txt")
        z.write(meta_path, arcname="meta.json")
        if audio_path and os.path.exists(audio_path):
            z.write(audio_path, arcname="summary_audio.mp3")

    return str(zip_path)

# ---- Minimal CSS vars for a clean light/dark toggle (no reload) ----
CUSTOM_CSS = """
:root {
  --app-radius: 14px;
  --app-pad: 14px;
}
.gradio-container { max-width: 1040px !important; margin: auto !important; }

.dark, .dark * {
  /* Dark mode overrides */
  --block-background-fill: #0b0f19 !important;
  --body-background-fill: #0b0f19 !important;
  --body-text-color: #e6e7eb !important;
  --link-text-color: #a5b4fc !important;
  --button-primary-background-fill: #1f2937 !important;
  --button-primary-text-color: #e6e7eb !important;
  --input-background-fill: #111827 !important;
  --input-border-color: #374151 !important;
  --border-color-primary: #374151 !important;
}
.dark .gradio-container, .dark .gr-block, .dark .gr-panel, .dark .gradio-container * {
  color: var(--body-text-color) !important;
}
.dark textarea, .dark input, .dark .tokenizer, .dark .wrap, .dark .form, .dark .prose {
  background: var(--input-background-fill) !important; color: var(--body-text-color) !important;
}
.card {
  border-radius: var(--app-radius);
  padding: var(--app-pad);
  border: 1px solid var(--border-color-primary);
  background: var(--block-background-fill);
}
"""

# ---- Build UI ----
with gr.Blocks(title="HF Summarizer + TTS", css=CUSTOM_CSS, fill_height=True) as demo:
    gr.Markdown("## 🔎 Text Summarization (Hugging Face) · 🔊 TTS · 💾 Export · 🌗 Dark/Light")

    with gr.Row():
        with gr.Column(scale=3):
            model_id = gr.Textbox(
                label="Hugging Face Model ID",
                value="facebook/bart-large-cnn",
                placeholder="e.g. facebook/bart-large-cnn, google/pegasus-xsum, philschmid/bart-large-cnn-samsum"
            )
        with gr.Column(scale=1):
            load_btn = gr.Button("Load / Reload Model", variant="primary")
            model_status = gr.Markdown("ℹ️ No model loaded yet.")
        with gr.Column(scale=1, min_width=160):
            dark_toggle = gr.Checkbox(label="🌗 Dark Mode", value=False, interactive=True)

    with gr.Row():
        with gr.Column():
            gr.Markdown("#### Input")
            input_text = gr.Textbox(lines=12, placeholder="Paste or type the text you want summarized...", container=True)
            with gr.Row():
                min_len = gr.Slider(10, 200, value=32, step=1, label="min_length")
                max_len = gr.Slider(50, 512, value=128, step=1, label="max_length")
            with gr.Row():
                do_sample = gr.Checkbox(value=False, label="do_sample")
                temperature = gr.Slider(0.1, 2.0, value=1.0, step=0.1, label="temperature")
                top_p = gr.Slider(0.1, 1.0, value=1.0, step=0.05, label="top_p")
            run_btn = gr.Button("▶️ Summarize", variant="primary")

        with gr.Column():
            gr.Markdown("#### Output")
            summary_out = gr.Textbox(lines=12, label="Summary", interactive=False)
            audio_out = gr.Audio(label="Summary Audio (TTS)", autoplay=False)
            with gr.Row():
                speak_btn = gr.Button("🔊 Read Summary")
                export_btn = gr.Button("💾 Export Summary + Audio (ZIP)")
            export_file = gr.File(label="Download Export (ZIP)", file_types=[".zip"])

    # Keep last audio path in state so export can include it
    audio_state = gr.State("")

    # --- Wire events ---
    load_btn.click(
        fn=_load_model,
        inputs=[model_id],
        outputs=[model_status, gr.Textbox(visible=False)],
        api_name="load_model"
    )

    def _do_all(text, max_length, min_length, do_sample, temperature, top_p, current_model):
        summary = _summarize(text, max_length, min_length, do_sample, temperature, top_p)
        # Create audio immediately after summarization for a smoother UX
        audio = _tts(summary, lang="en")
        return summary, audio, audio

    run_btn.click(
        fn=_do_all,
        inputs=[input_text, max_len, min_len, do_sample, temperature, top_p, model_id],
        outputs=[summary_out, audio_out, audio_state],
        api_name="summarize"
    )

    speak_btn.click(
        fn=lambda s: _tts(s, lang="en"),
        inputs=[summary_out],
        outputs=[audio_out],
        api_name="tts"
    )

    export_btn.click(
        fn=_export,
        inputs=[input_text, summary_out, model_id, audio_state],
        outputs=[export_file],
        api_name="export"
    )

    # Dark/Light toggle via simple JS; no reload needed
    # (Just flips a 'dark' class on <html>)
    dark_toggle.change(
        fn=lambda x: None,  # no Python work
        inputs=[dark_toggle],
        outputs=[],
        js="""
            (checked) => {
              const root = document.documentElement;
              if (checked) root.classList.add('dark');
              else root.classList.remove('dark');
            }
        """,
    )

demo.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.2 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://34da67ef173cd8ee28.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
